In [ ]:
# Plotting churn rate across categorical features
plt.figure(figsize=(15, 25))

for i, col in enumerate(categorical_cols, 1):
    plt.subplot(6, 3, i)
    churn_rate = data.groupby(col)['churn'].value_counts(normalize=True).unstack()['Yes']
    churn_rate.plot(kind='bar', color='salmon', edgecolor='black')
    plt.title(f'Churn Rate in {col.capitalize()}')
    plt.ylabel('Churn Rate')
    plt.xlabel(col.capitalize())

plt.tight_layout()
plt.show()


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

# Load the dataset
data = pd.read_csv('cleaned_data_telecom.csv')  # Update path if needed

# Define the feature columns and target column
features = data.columns.drop('churn')
target = 'churn'

# Step 1: Split the data into training (60%), validation (20%), and testing (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(data[features], data[target], test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define categorical and numerical features
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Step 2: Define separate transformers for each set
# Transformer for training set
train_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)])

# Transformer for validation set
val_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)])

# Transformer for test set
test_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)])

# Step 3: Fit each preprocessor only on its respective set and transform
X_train_preprocessed = train_preprocessor.fit_transform(X_train)
X_val_preprocessed = val_preprocessor.fit_transform(X_val)
X_test_preprocessed = test_preprocessor.fit_transform(X_test)

# Display shapes of the preprocessed sets to confirm successful transformation
print("Training set shape:", X_train_preprocessed.shape)
print("Validation set shape:", X_val_preprocessed.shape)
print("Test set shape:", X_test_preprocessed.shape)


Training set shape: (4206, 30)
Validation set shape: (1402, 30)
Test set shape: (1402, 30)


In [7]:
data = pd.read_csv('cleaned_data_telecom.csv')

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def preprocess_and_split_data(df, target_column, test_size=0.4, val_size=0.5, random_state=42):
    """
    Split the data into training, validation, and testing sets, and preprocess each set with scaling and encoding.
    
    Parameters:
    df (pd.DataFrame): The dataset to split and preprocess.
    target_column (str): The name of the target column.
    test_size (float): The proportion of data to include in the test + validation sets.
    val_size (float): The proportion of the remaining data (test + validation) to allocate to validation.
    random_state (int): Seed for random splitting.
    
    Returns:
    tuple: Preprocessed training, validation, and test sets (X_train, X_val, X_test, y_train, y_val, y_test).
    """
    # Step 1: Define features and split the data
    features = df.columns.drop(target_column)
    X = df[features]
    y = df[target_column]
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=random_state)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=val_size, random_state=random_state)
    
    # Step 2: Identify categorical and numerical features
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
    
    # Step 3: Define and apply transformers for each set
    train_preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(drop='first'), categorical_features)])
    
    val_preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(drop='first'), categorical_features)])
    
    test_preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(drop='first'), categorical_features)])
    
    # Fit each preprocessor only on its respective set and transform
    X_train_preprocessed = train_preprocessor.fit_transform(X_train)
    X_val_preprocessed = val_preprocessor.fit_transform(X_val)
    X_test_preprocessed = test_preprocessor.fit_transform(X_test)
    
    # Display shapes of the preprocessed sets to confirm successful transformation
    print("Training set shape:", X_train_preprocessed.shape)
    print("Validation set shape:", X_val_preprocessed.shape)
    print("Test set shape:", X_test_preprocessed.shape)
    
    return X_train_preprocessed, X_val_preprocessed, X_test_preprocessed, y_train, y_val, y_test

# Example usage:
# Assuming `data` is your DataFrame and 'churn' is the target column
X_train_preprocessed, X_val_preprocessed, X_test_preprocessed, y_train, y_val, y_test = preprocess_and_split_data(data, target_column='churn')


Training set shape: (4206, 30)
Validation set shape: (1402, 30)
Test set shape: (1402, 30)


In [34]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize logistic regression model
log_reg = LogisticRegression(max_iter=1000)

# Train the model on the preprocessed training set
log_reg.fit(X_train_preprocessed, y_train)

# Extract and display coefficients with feature names
coefficients = pd.DataFrame({
    'Feature': train_preprocessor.get_feature_names_out(),
    'Coefficient': log_reg.coef_[0]
})

# Sort coefficients by their absolute value to see the most influential features
coefficients['Abs_Coefficient'] = coefficients['Coefficient'].abs()
coefficients = coefficients.sort_values(by='Abs_Coefficient', ascending=False).drop(columns=['Abs_Coefficient'])

print("\nLogistic Regression Coefficients:")
print(coefficients)

# Validate the model on the validation set (optional)
y_val_pred = log_reg.predict(X_val_preprocessed)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"\nValidation Accuracy: {val_accuracy * 100:.2f}%")
print("\nValidation Set Classification Report:")
print(classification_report(y_val, y_val_pred))



Logistic Regression Coefficients:
                                        Feature  Coefficient
1                                   num__tenure    -1.479676
25                       cat__contract_Two year    -1.385469
3                            num__total_charges     0.763472
10            cat__internet_service_Fiber optic     0.745609
24                       cat__contract_One year    -0.682765
7                        cat__phone_service_Yes    -0.496263
13                     cat__online_security_Yes    -0.412952
9                       cat__multiple_lines_Yes     0.357369
26                   cat__paperless_billing_Yes     0.346578
19                        cat__tech_support_Yes    -0.288973
23                    cat__streaming_movies_Yes     0.211951
6                           cat__dependents_Yes    -0.210361
27  cat__payment_method_Credit card (automatic)    -0.196227
21                       cat__streaming_t_v_Yes     0.193864
28         cat__payment_method_Electronic check   

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Filter data for customers with tenure <= 1 month
short_term_customers = data[data['tenure'] <= 1].copy()

# Step 2: Define features and target variable
features = ['contract', 'internet_service', 'online_security', 'tech_support', 
            'streaming_t_v', 'payment_method']  # Add relevant features
target = 'churn'

# Convert categorical features using one-hot encoding
short_term_customers_encoded = pd.get_dummies(short_term_customers[features], drop_first=True)
X = short_term_customers_encoded
y = short_term_customers[target].apply(lambda x: 1 if x == 'Yes' else 0)

# Step 3: Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Train Decision Tree model
tree_clf = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_clf.fit(X_train, y_train)

# Step 5: Evaluate Logistic Regression on validation set
y_val_pred_log_reg = log_reg.predict(X_val)
log_reg_accuracy = accuracy_score(y_val, y_val_pred_log_reg)
print(f"Logistic Regression Validation Accuracy: {log_reg_accuracy * 100:.2f}%")
print("\nLogistic Regression Classification Report:")
print(classification_report(y_val, y_val_pred_log_reg))

# Step 5: Evaluate Decision Tree on validation set
y_val_pred_tree = tree_clf.predict(X_val)
tree_accuracy = accuracy_score(y_val, y_val_pred_tree)
print(f"Decision Tree Validation Accuracy: {tree_accuracy * 100:.2f}%")
print("\nDecision Tree Classification Report:")
print(classification_report(y_val, y_val_pred_tree))

# Optional: Feature Importance for Decision Tree
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': tree_clf.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature Importances from Decision Tree:")
print(feature_importances)


In [7]:
data.columns

Index(['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_t_v', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges', 'churn'],
      dtype='object')

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Filter data for customers with tenure <= 1 month
short_term_customers = data[data['tenure'] <= 1].copy()

# Step 2: Define features and target variable
features = ['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_t_v', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges']  # Add relevant features
target = 'churn'

# Convert categorical features using one-hot encoding
short_term_customers_encoded = pd.get_dummies(short_term_customers[features], drop_first=True)
X = short_term_customers_encoded
y = short_term_customers[target].apply(lambda x: 1 if x == 'Yes' else 0)

# Step 3: Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Train Decision Tree model
tree_clf = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_clf.fit(X_train, y_train)

# Step 5: Evaluate Logistic Regression on validation set
y_val_pred_log_reg = log_reg.predict(X_val)
log_reg_accuracy = accuracy_score(y_val, y_val_pred_log_reg)
print(f"Logistic Regression Validation Accuracy: {log_reg_accuracy * 100:.2f}%")
print("\nLogistic Regression Classification Report:")
print(classification_report(y_val, y_val_pred_log_reg))

# Step 5: Evaluate Decision Tree on validation set
y_val_pred_tree = tree_clf.predict(X_val)
tree_accuracy = accuracy_score(y_val, y_val_pred_tree)
print(f"Decision Tree Validation Accuracy: {tree_accuracy * 100:.2f}%")
print("\nDecision Tree Classification Report:")
print(classification_report(y_val, y_val_pred_tree))

# Optional: Feature Importance for Decision Tree
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': tree_clf.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature Importances from Decision Tree:")
print(feature_importances)


Logistic Regression Validation Accuracy: 67.42%

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.61      0.61        75
           1       0.72      0.72      0.72       103

    accuracy                           0.67       178
   macro avg       0.67      0.67      0.67       178
weighted avg       0.67      0.67      0.67       178

Decision Tree Validation Accuracy: 66.29%

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.75      0.65        75
           1       0.77      0.60      0.67       103

    accuracy                           0.66       178
   macro avg       0.67      0.67      0.66       178
weighted avg       0.69      0.66      0.66       178


Feature Importances from Decision Tree:
                                   Feature  Importance
10            internet_service_Fiber optic    0.554301
2                      